# PJN - Lab 9 RAG Advanced

#### Mikołaj Stefański

## ETAP I — dynamiczny retrieval adaptacyjny

### Zadanie 1 – analiza zapytania i zmiana wagi hybrydy

In [ ]:
import sys
import os

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

from rag.reasoning.query_analysis import analyze_query_intent
from rag.retrieval.search_engine import hybrid_search

test_queries = [
    "Jak poprawić pracę zespołową?",          
    "Co zawiera dokument PAN LP-78?",          
    "Czy inflacja 2022 była wyższa?",          
    "Co Sinek mówił o liderach?",             
    "Wydarzenia klimatyczne w Polsce",         
    "Protokół HTTP błędy 404",                 
    "Definicja empatii",                       
    "Raport ONZ 2023",                        
]

print(f"{'ZAPYTANIE':<40} | {'TRYB':<10} | {'ES WAGA':<8} | {'QDRANT':<8}")
print("-" * 80)

for query in test_queries:
    config = analyze_query_intent(query)
    
    try:
        q_res, es_res, hybrid_res = hybrid_search(
            query, 
            limit=3, 
            weight_es=config['es_weight'], 
            weight_qdrant=config['qdrant_weight']
        )
        
        print(f"{query:<40} | {config['mode'].upper():<10} | {config['es_weight']:<8} | {config['qdrant_weight']:<8}")
        
        if hybrid_res:
            top_text = hybrid_res[0]['text'][:50].replace('\n', ' ')
            
    except Exception as e:
        print(f"Błąd przy zapytaniu '{query}': {e}")

ZAPYTANIE                                | TRYB       | ES WAGA  | QDRANT  
--------------------------------------------------------------------------------
Jak poprawić pracę zespołową?            | SEMANTIC   | 0.4      | 0.6     
Co zawiera dokument PAN LP-78?           | FACTUAL    | 0.8      | 0.2     
Czy inflacja 2022 była wyższa?           | FACTUAL    | 0.8      | 0.2     
Co Sinek mówił o liderach?               | SEMANTIC   | 0.4      | 0.6     
Wydarzenia klimatyczne w Polsce          | SEMANTIC   | 0.4      | 0.6     
Protokół HTTP błędy 404                  | FACTUAL    | 0.8      | 0.2     
Definicja empatii                        | SEMANTIC   | 0.4      | 0.6     
Raport ONZ 2023                          | FACTUAL    | 0.8      | 0.2     


#### Wyniki

System poprawnie rozpoznaje typ zapytania. Pytania z akronimami czy liczbami klasyfikuje jako `FACTUAL`, a pytania abstrakcyjne jako `SEMANTIC`.

### Zadanie 2

In [3]:
from rag.reasoning.filtering import filter_retrieved_docs
from rag.retrieval.search_engine import hybrid_search

query = "PAN" 

print(f"Test filtrowania dla zapytania: '{query}'.\n")

_, _, raw_results = hybrid_search(query, limit=10)

print(f"Surowe dane z bazy: {len(raw_results)}")

clean_docs, stats = filter_retrieved_docs(raw_results, min_words=30, max_docs=3)

print("\nStatystyki filtra:")
print(f" -> Odrzucono - za krótkie: {stats['rejected_short']}")
print(f" -> Współczynnik odrzuceń: {stats['rejected_ratio']}")
print(f" -> Wynik końcowy: {stats['final_count']} dokumentów")

print("\nOdfiltrowane dokumenty:")
for i, doc in enumerate(clean_docs, 1):
    preview = doc['text'][:80].replace('\n', ' ')
    print(f" {i}. [{len(doc['text'].split())} słów] {preview}.")

Test filtrowania dla zapytania: 'PAN'.

Surowe dane z bazy: 10

Statystyki filtra:
 -> Odrzucono - za krótkie: 7
 -> Współczynnik odrzuceń: 70.0%
 -> Wynik końcowy: 3 dokumentów

Odfiltrowane dokumenty:
 1. [47 słów] Wśród laureatów znajdują się m.in. prof. dr hab. Barbara Bilińska, członek kores.
 2. [35 słów] I powiedział pewien człowiek wierzący, z rodu Faraona, który ukrywał swoją wiarę.
 3. [47 słów] Wśród laureatów znajdują się m.in. prof. dr hab. Barbara Bilińska, członek kores.


#### Wyniki

Odrzucenie 7 tekstów oszczędza czas i zasoby potrzebne do ich przetworzenia przez LLM. Model nie będzie próbował zgadnąć odpowiedzi na podstawie urwanego nagłówka czy stopki zawierającej słowo 'PAN'. 

## Etap II — Query Decomposition + Query Rewriting

### Zadanie 1

In [ ]:
from rag.reasoning.query_analysis import decompose_query, generate_clarification_question

complex_queries = [
    "Jak poprawić pracę zespołową?",           
    "Jakie dane o klimacie po 2020 w Polsce?",
    "Co Sinek mówił o liderach?",              
    "Czy inflacja rośnie?"                    
    ]

print("Rozpoczynam test dekompozycji.\n")

for q in complex_queries:
    print(f"Zapytanie główne: '{q}'")
    
    result = decompose_query(q)
    
    print(" -> Podpytania wygenerowane przez AI:")
    for sub_q in result.get('sub_questions', []):
        print(f"    * {sub_q}")
    print("=" * 60)

Rozpoczynam test dekompozycji.

Zapytanie główne: 'Jak poprawić pracę zespołową?'
 -> Podpytania wygenerowane przez AI:
    * Jakie są kluczowe elementy dobrej pracy w zespole?
    * Co wpływa na efektywność zespołu?
Zapytanie główne: 'Jakie dane o klimacie po 2020 w Polsce?'
 -> Podpytania wygenerowane przez AI:
    * Czy istnieje dostępna publiczna baza danych z danymi klimatycznymi dla Polski po 2020 roku?
    * Jakie konkretne dane (np. temperatura, wilgotność, opady) interesują Cię najbardziej?
Zapytanie główne: 'Co Sinek mówił o liderach?'
 -> Podpytania wygenerowane przez AI:
    * Jakie informacje na temat liderów podał Sinek?
    * W jakim kontekście Sinek omawiał liderów?
Zapytanie główne: 'Czy inflacja rośnie?'
 -> Podpytania wygenerowane przez AI:
    * Jaka jest aktualna inflacja w Polsce?
    * Jakie są trendy inflacji w ostatnich miesiącach? 


#### Wyniki

Model Gemma dobrze poradził sobie z dekompozycją. Zamiast odpowiadać tak lub nie, pyta o "aktualną inflację" i "trendy". To bardzo dobry punkt wyjścia dla naszego systemu, bo pozwala pobrać konkretne dane liczbowe, natomiast rozbicie na "kluczowe elementy" i "efektywność" pozwoli systemowi wyszukać szerszy kontekst.

### Zadanie 2

In [4]:
from rag.reasoning.query_analysis import generate_clarification_question

ambiguous_queries = [
    "Co mówi PAN o kryzysie?",         
    "Jaki ma sens odpowiedzialność?",  
    "Jak zarządzać ludźmi?",           
    "Zamek w Polsce"                   
]

print("Rozpoczynam test pytań doprecyzowujących.\n")

for q in ambiguous_queries:
    print(f"Zapytanie: '{q}'")
    
    result = generate_clarification_question(q)
    
    is_ambiguous = result.get('is_ambiguous', False)
    print(f" -> Czy jest niejednoznaczne? {'TAK' if is_ambiguous else 'NIE'}")
    
    if is_ambiguous:
        print(" -> Proponowane interpretacje:")
        for interp in result.get('interpretations', []):
            print(f"    * {interp}")
            
    print("=" * 60)

Rozpoczynam test pytań doprecyzowujących.

Zapytanie: 'Co mówi PAN o kryzysie?'
 -> Czy jest niejednoznaczne? TAK
 -> Proponowane interpretacje:
    * Co Pan profesor z Polskiej Akademii Nauk powiedział o kryzysie?
    * Czy PAN komentował kryzys w jakimś wypowiedzi?
Zapytanie: 'Jaki ma sens odpowiedzialność?'
 -> Czy jest niejednoznaczne? TAK
 -> Proponowane interpretacje:
    * Jaki ma sens **wymiar odpowiedzialności**?
    * Jaki ma sens **poziom odpowiedzialności w danej sytuacji**?
Zapytanie: 'Jak zarządzać ludźmi?'
 -> Czy jest niejednoznaczne? TAK
 -> Proponowane interpretacje:
    * **Interpretacja 1:** Jak zarządzać ludźmi w kontekście zarządzania zespołem, jak to zrobić w pracy, by zespół był efektywny i produktywny?
    * **Interpretacja 2:** Jak zarządzać ludźmi w kontekście relacji interpersonalnych, jak budować zdrowe i szczęśliwe relacje z innymi ludźmi?
Zapytanie: 'Zamek w Polsce'
 -> Czy jest niejednoznaczne? TAK
 -> Proponowane interpretacje:
    * Zamek w Polsce jako

#### Wyniki

1. Co mówi PAN o kryzysie?

    Model słusznie powiązał akronim "PAN" z Polską Akademią Nauk, odróżniając go od zwrotu grzecznościowego "Pan". ES zwróciłby zarówno dokumenty o naukowcach, jak i cytaty z Biblii ("Pan Bóg"). Dzięki klaryfikacji system wie, że użytkownik szuka opinii eksperckiej/instytucjonalnej.

2. Jak zarządzać ludźmi?

    Model doskonale rozdzielił kontekst zawodowy od interpersonalnego. To dwa rozdzielne zbiory dokumentów. Bez tego pytania RAG mógłby mieszać porady biznesowe z poradnikami psychologicznymi, tworząc niespójną odpowiedź.

3. Zamek w Polsce.

    Rozróżnienie między konkretną encją, a tematem ogólnym. Pozwala systemowi zdecydować, czy szukać listy zamków, czy definicji/historii.

4. Jaki ma sens odpowiedzialność?

    Interpretacje są dość abstrakcyjne. Zabrakło wyraźnego rozróżnienia np. na odpowiedzialność prawną vs moralną. Mniejsza niż w pozostałych, ale nadal sygnalizuje użytkownikowi, że pytanie jest zbyt ogólne.



| Zapytanie                      | Przydatność                          |
| ------------------------------ | ------------------------------------ |
| Co mówi PAN o kryzysie?        | Duża                                 |
| Jak zarządzać ludźmi?          | Bardzo duża                          |
| Zamek w Polsce.                | Duża                                 |
| Jaki ma sens odpowiedzialność? | Średnia                              |


## Etap III

In [1]:
from rag.memory.memory_manager import memory

trudne_pytanie = "Jaki jest kod błędu X-99 w reaktorze?"

print(f"Rozpoczynam test pamięci systemowej.\n")
print(f"Zapytanie: '{trudne_pytanie}'")

memory.add_pending_query(trudne_pytanie, reason="brak dokumentów w bazie")

pending_list = memory.get_pending_queries()

print("\nZawartość pliku pending.json:")
for item in pending_list:
    print(f" [ID: {item['id']}] Status: {item['status'].upper()} | Pytanie: {item['query']}")

Rozpoczynam test pamięci systemowej.

Zapytanie: 'Jaki jest kod błędu X-99 w reaktorze?'
Odłożono pytanie do wyjaśnienia: 'Jaki jest kod błędu X-99 w reaktorze?'

Zawartość pliku pending.json:
 [ID: 1] Status: PENDING | Pytanie: Jaki jest kod błędu X-99 w reaktorze?


#### Wyniki

Plik pending.json został utworzony, a system poprawnie zapisał pytanie, na które nie znalazł odpowiedzi. Mechanizm pamięci działa. Zatem został zbudowany fundament, żeby RAG nie musiał wymyślać odpowiedzi, gdy czegoś nie wie.

## Etap IV — warstwa weryfikacji

### Zadanie 1

In [1]:
from rag.verification.validator import validate_rag_answer, safe_mode_retry

print("Rozpoczynam test walidatora.\n")

query_1 = "Czym jest praca zespołowa?"
context_1 = [{"text": "Praca zespołowa to skoordynowane działania grupy osób w celu osiągnięcia wspólnego celu."}]
answer_1 = "To działanie grupy ludzi dążących do wspólnego celu."

print(f"Test 1: Pytanie: '{query_1}'")
valid_1, reason_1 = validate_rag_answer(query_1, answer_1, context_1)
print(f" -> Czy poprawne? {valid_1}")
print(f" -> Wyjaśnienie: {reason_1}")
print("=" * 60)

query_2 = "Jaki jest ulubiony kolor prezesa?"
context_2 = [{"text": "Prezes zarządza firmą i odpowiada za strategię finansową spółki."}] 
answer_2 = "Ulubionym kolorem prezesa jest niebieski."

print(f"Test 2: Pytanie: '{query_2}'")
valid_2, reason_2 = validate_rag_answer(query_2, answer_2, context_2)

print(f" -> Czy poprawne? {valid_2}") 
print(f" -> Wyjaśnienie: {reason_2}")

if not valid_2:
    print("\nWykryto halucynację. Uruchamiam Safe Mode.")
    safe_response = safe_mode_retry(query_2, strategy="memory")
    print(f"Odpowiedź dla użytkownika: {safe_response}")

Rozpoczynam test walidatora.

Test 1: Pytanie: 'Czym jest praca zespołowa?'
 -> Czy poprawne? True
 -> Wyjaśnienie: Odpowiedź systemu zawiera poprawne definicje pracy zespołowej. W źródłach wskazano, że praca zespołowa to działanie grupy osób w celu osiągnięcia wspólnego celu. Takie definicje są zgodne z zasadami pracy zespołowej.
Test 2: Pytanie: 'Jaki jest ulubiony kolor prezesa?'
 -> Czy poprawne? False
 -> Wyjaśnienie: Dostępne źródła nie dostarczają informacji o ulubionym kolorze prezesa.

Wykryto halucynację. Uruchamiam Safe Mode.
Odpowiedź dla użytkownika: Nie jestem pewien odpowiedzi na podstawie dostępnych dokumentów. Pytanie zostało zapisane do wyjaśnienia przez eksperta.


#### Wyniki

Logi wykazują, że walidator działa poprawnie:

- Test 1: walidator zrozumiał kontekst, przeczytał definicję i potwierdził poprawność.
- Test 2: walidator zauważył brak informacji w źródłach, odrzucił odpowiedź i uruchomił Safe Mode, który zapisał pytnie do pamięci.

#### Pytanie: Jaki jest cel podobnego działania?

1. Ochrona autorytetu: najgorsze co RAG może zrobić, to z pełnym przekonaniem podać nieprawdę. Safe Mode to hamulec bezpieczeństwa - lepiej żeby system powiedział, że nie znalazł informacji na ten temat, niż żeby zaczął zmyślać odpowiedź.

2. Domykanie pętli: zapisanie pytania do pending.json pozwala człowiekowi zobaczyć, czego system nie wie. Można wtedy dodać brakujący dokument do bazy i następnym razem system już odpowie poprawnie.

3. Autonomiczna naprawa: strategie takie jak modify_prompt pozwalają systemowi spróbować jeszcze raz, w inny sposób, zanim się podda. To symuluje ludzkie zachowanie – jak nie znajdzie się czegoś w internecie za pierwszym razem, zmieniamy słowa kluczowe.

### Integracja systemu

In [2]:
from rag_pipeline import run_rag_pipeline

print("="*80)
print("Test 1: Pytanie poprawne.")
print("="*80)

query_a = "Kim jest Barbara Bilińska?"
response_a = run_rag_pipeline(query_a)

print(f"\nFinalna odpowiedź dla użytkownika:\n{response_a}\n")


print("\n" + "="*80)
print("Test 2: Celowa halucynacja.")
print("="*80)

query_b = "Podaj nazwę ulubionego koloru prezesa PAN."
response_b = run_rag_pipeline(query_b)

print(f"\nFinalna odpowiedź dla użytkownika:\n{response_b}\n")


Test 1: Pytanie poprawne.

Zaczynamy! Pytanie użytkownika: 'Kim jest Barbara Bilińska?'
Wykryto niejednoznaczność, ale kontynuuję na potrzeby testu.
   -> System zapytałby: Kim jest Barbara Bilińska jako osoba publiczna?...
Rozpoczynam analizę intencji. Tryb: SEMANTIC. Wagi -> ES: 0.4, Qdrant: 0.6
Rozpoczynam filtrowanie. Wybrano: 4 dokumentów. Odrzucono krótkie: 0
Generowanie odpowiedzi.
   -> Wstępny draft: Prof. dr hab. Barbara Bilińska jest członkiem korespondent PAN....
Weryfikacja faktów.
Zweryfikowano pomyślnie.

Finalna odpowiedź dla użytkownika:
Prof. dr hab. Barbara Bilińska jest członkiem korespondent PAN.


Test 2: Celowa halucynacja.

Zaczynamy! Pytanie użytkownika: 'Podaj nazwę ulubionego koloru prezesa PAN.'
Wykryto niejednoznaczność, ale kontynuuję na potrzeby testu.
   -> System zapytałby: Nazwa ulubionego koloru prezesa PAN w kontekście jego osobistej preferencji barw...
Rozpoczynam analizę intencji. Tryb: FACTUAL. Wagi -> ES: 0.8, Qdrant: 0.2
Rozpoczynam filtrowanie.

#### Wyniki

Przeprowadzone testy potwierdziły poprawność działania zaimplementowanego potoku RAG. System wykazał się odpornością na halucynacje oraz zdolnością do dynamicznego dostosowywania strategii wyszukiwania. Poniżej szczegółowa analiza kluczowych modułów:

1. **Skuteczność warstwy weryfikacji**

    Najważniejszym osiągnięciem jest skuteczna eliminacja halucynacji w teście nr 2. Standardowy RAG często zmyśla odpowiedź, próbując na siłę dopasować kontekst. Zaimplementowany moduł `validator` oparty na modelu gemma2:2b poprawnie zidentyfikował brak informacji o "kolorze prezesa" w dostarczonych dokumentach. Mechanizm ten działa jako krytyczny bezpiecznik, zmieniając system z generatora tekstu w wiarygodnego asystenta, który potrafi przyznać się do niewiedzy.

2. **Dynamiczne sterowanie wyszukiwaniem**

    System poprawnie rozpoznał intencje użytkownika, co widać w logach doboru wag:

    - w przypadku pytania o definicję/osobę w teście: System zaklasyfikował intencję jako SEMANTIC (wagi: ES 0.4 / Qdrant 0.6), co pozwoliło na szersze, wektorowe dopasowanie kontekstu.

    - w przypadku pytania o konkretny fakt w teście 2: system zmienił tryb na FACTUAL (wagi: ES 0.8 / Qdrant 0.2), priorytetyzując słowa kluczowe. Dowodzi to, że system nie traktuje każdego zapytania tak samo, lecz optymalizuje strategię retrievalu pod kątem specyfiki pytania.

3. **Mechanizm "Safe Mode" i pamięć systemu**

    Zamiast zwracać pusty błąd lub nieprawdziwą informację, system uruchomił procedurę Safe Mode. Zapisanie nierozwiązanego pytania do pending_queries.json stanowi fundament pod tzw. "Human-in-the-loop". W środowisku produkcyjnym pozwala to administratorom identyfikować luki w bazie wiedzy i je uzupełniać, co sprawia, że system uczy się na swoich porażkach.

4. **Nadwrażliwość funkcji klaryfikacji**
 
    W obu testach moduł klaryfikacji uznał zapytania za niejednoznaczne, mimo że były one stosunkowo precyzyjne. Model o mniejszej liczbie parametrów taki jak `gemma2:2b` ma tendencję do nadinterpretacji instrukcji. W przyszłych iteracjach należałoby albo dostroić prompt za pomocą few-shot prompting z przykładami pytań jednoznacznych, albo zastosować silniejszy model do tego konkretnego zadania np. model 7B. Mimo to, zachowanie systemu - ostrzeżenie zamiast blokady - było akceptowalne.

5. **Podsumowanie**

    Advanced RAG zrealizował założenia architektury odpornej na błędy. Połączenie wyszukiwania hybrydowego, analizy intencji oraz weryfikacji faktów pozwoliło uzyskać system, który precyzyjnie odpowiedzia na pytanie z bazy i skutecznie obronił się przed wygenerowaniem fałszywej informacji.

## ETAP V — benchmark jakości RAG-a

In [4]:
from rag_pipeline import run_rag_pipeline
import pandas as pd

benchmark_data = [
    {
        "Zapytanie": "Kim był Jan Paweł II?",
        "Oczekiwane": "Oczekiwana poprawna odpowiedź z bazy."
    },
    {
        "Zapytanie": "Kim jest Howard W. Campbell?",
        "Oczekiwane": "Oczekiwana poprawna odpowiedź."
    },
    {
        "Zapytanie": "Co to jest Światowy Dzień Wody?",
        "Oczekiwane": "Oczekiwana definicja lub data z bazy."
    },
    {
        "Zapytanie": "Kim jest Barbara Bilińska?",
        "Oczekiwane": "Oczekiwana informacja o profesurze/PAN."
    },

    {
        "Zapytanie": "jak wzrost cen zmienia płace", 
        "Oczekiwane": "Brak danych - Safe Mode."
    },
    {
        "Zapytanie": "ponoszenie konsekwencji za swoje błędy",
        "Oczekiwane": "Brak danych - Safe Mode."
    },

    {
        "Zapytanie": "Co mówi PAN o kryzysie?",
        "Oczekiwane": "Wykrycie niejednoznaczności."
    },
    {
        "Zapytanie": "najlepsze filmy 2024",
        "Oczekiwane": "Brak danych - Safe Mode."
    }
]

results_list = []

for item in benchmark_data:
    print(f"Zapytanie: '{item['Zapytanie']}'")
    
    response = run_rag_pipeline(item['Zapytanie'])
    
    short_response = response[:100] + "..." if len(response) > 100 else response
    results_list.append({
        "Zapytanie": item['Zapytanie'],
        "Odpowiedź": short_response
    })
    print("-" * 50)

Zapytanie: 'Kim był Jan Paweł II?'

Zaczynamy! Pytanie użytkownika: 'Kim był Jan Paweł II?'
Wykryto niejednoznaczność, ale kontynuuję na potrzeby testu.
   -> System zapytałby: Kim był Jan Paweł II w kontekście jego roli jako papież?...
Rozpoczynam analizę intencji. Tryb: FACTUAL. Wagi -> ES: 0.8, Qdrant: 0.2
Rozpoczynam filtrowanie. Wybrano: 4 dokumentów. Odrzucono krótkie: 0
Generowanie odpowiedzi.
   -> Wstępny draft: Jan Paweł II był papieżem Kościoła katolickiego....
Weryfikacja faktów.
Zweryfikowano pomyślnie.
--------------------------------------------------
Zapytanie: 'Kim jest Howard W. Campbell?'

Zaczynamy! Pytanie użytkownika: 'Kim jest Howard W. Campbell?'
Wykryto niejednoznaczność, ale kontynuuję na potrzeby testu.
   -> System zapytałby: Kim jest Howard W. Campbell jako osoba?...
Rozpoczynam analizę intencji. Tryb: SEMANTIC. Wagi -> ES: 0.4, Qdrant: 0.6
Rozpoczynam filtrowanie. Wybrano: 4 dokumentów. Odrzucono krótkie: 0
Generowanie odpowiedzi.
   -> Wstępny draft: Howa

#### Tabela

| Zapytanie | Typ | Cytuje źródło? | Halucynacja? | Ocena (0–2) | Komentarz / Uzasadnienie |
| :--- | :--- | :---: | :---: | :---: | :--- |
| **"Kim był Jan Paweł II?"** | Dane z bazy | **TAK** | **NIE** | **2** | System poprawnie zidentyfikował papieża na podstawie bazy. Pełny sukces pipeline'u. |
| **"Kim jest Barbara Bilińska?"** | Dane z bazy | **TAK** | **NIE** | **2** | Prawidłowa identyfikacja członkini PAN. Walidacja pozytywna. |
| **"Kim jest Howard W. Campbell?"** | Dane z bazy | **NIE** | **NIE** | **1** | Dokument istnieje w bazie, Generator stworzył poprawny draft, ale Walidator go odrzucił. System zadziałał zbyt ostrożnie. |
| **"Co to jest Światowy Dzień Wody?"** | Dane z bazy | **NIE** | **NIE** | **2** | Dokument w bazie jest zbyt ogólnikowy ("istotne wydarzenie"). System słusznie uznał, że to za mało na definicję i uruchomił Safe Mode. |
| **"jak wzrost cen zmienia płace"** | Lab 6 | **NIE** | **NIE** | **2** | Brak danych ekonomicznych w korpusie. System odmówił spekulacji. |
| **"ponoszenie konsekwencji..."** | Lab 7 | **NIE** | **NIE** | **2** | Brak odpowiedniego kontekstu. Prawidłowy Safe Mode. |
| **"Co mówi PAN o kryzysie?"** | Decomposition | **N/D** | **NIE** | **2** | Poprawna detekcja niejednoznaczności i brak wiedzy o kryzysie. Safe Mode. |
| **"najlepsze filmy 2024"** | Spoza korpusu | **NIE** | **NIE** | **2** | System oparł się pokusie zmyślania listy filmów. Ocena maksymalna za uczciwość. |

#### Wyniki

**Analiza Przypadków**

Przeprowadzone testy wykazały, że zaimplementowana architektura priorytetyzuje wiarygodność i bezpieczeństwo danych. Dla znanych faktów precyzja jest wysoka: w przypadkach, gdy kontekst był jednoznaczny czyli Jan Paweł II i Barbara Bilińska, system bezbłędnie przechodził cały proces od wyszukiwania do walidacji. 
W przypadku zapytania o Howarda Campbella, system weryfikujący odrzucił poprawny draft odpowiedzi. Choć jest to błąd typu false negative, w systemach produkcyjnych jest on preferowany nad false positive czyli przepuszczeniem kłamstwa. Wskazuje to na konieczność użycia większego modelu w roli sędziego w przyszłych iteracjach.
Dla wszystkich pytań spoza domeny - ekonomia, filmy, ogólne definicje niedostępne w TOP-4 -  system konsekwentnie uruchamiał procedurę Safe Mode, zapisując zapytania do pamięci zamiast generować halucynacje.

Powyższe przypadki dowodzą, że system Advanced RAG skutecznie filtruje nie tylko halucynacje, ale także treści niskiej jakości, co jest pożądaną cechą.